In [ ]:
# pipsas

import numpy as np
import gymnasium as gym
import mo_gymnasium as mo_gym
from collections import defaultdict
import matplotlib.pyplot as plt

# abbv

# acc: acumulada
# est: estados
# eps: episodios
# obj: objetivos
# act: acciones

In [ ]:

env = mo_gym.make("mo-lunar-lander-v3")
num_act = env.action_space.n
num_est = 1000
alpha = 0.1
gamma = 0.99
epsilon = 0.1
eps = 500
num_obj = len(env.reward_range)

Q = defaultdict(lambda: np.zeros((num_act, num_obj)))
frontera_pareto = []


In [ ]:

def discret(estado):
    return tuple(np.round(estado, decimals=1))

def selc_accion(estado):
    if np.random.rand() < epsilon:
        return np.random.randint(num_act)
    else:
        valores = Q[estado]
        escal = valores @ np.ones(num_obj) # valores escalarizados
        return np.argmax(escal)

def act(frontera, nuevo_vec): # Pareto front, imagen del cto de Pareto 
    no_dominados = []
    for vec in frontera:
        if np.all(vec >= nuevo_vec) and np.any(vec > nuevo_vec):
            return frontera
        elif not (np.all(nuevo_vec >= vec) and np.any(nuevo_vec > vec)):
            no_dominados.append(vec)
    no_dominados.append(nuevo_vec)
    return no_dominados


In [ ]:

r_hist = []

for ep in range(eps):
    estado, _ = env.reset()
    estado = discret(estado)
    terminado = False
    r_acc = np.zeros(num_obj) # recompensa acc

    while not terminado:
        accion = selc_accion(estado)
        nv_estado, r_vec, fin, tronco, _ = env.step(accion)
        nv_estado = discret(nv_estado)
        terminado = fin or tronco

        mejor_sig = np.max(Q[nv_estado], axis=0)
        Q[estado][accion] = (1 - alpha) * Q[estado][accion] + alpha * (r_vec + gamma * mejor_sig)

        estado = nv_estado
        r_acc = r_acc + r_vec

    r_hist.append(r_acc)
    front = act(front, r_acc)

    if ep % 50 == 0:
        print(f"Episodio {ep}, Recompensa acumulada: {r_acc}")

env.close()


In [ ]:

r_dicc = np.array(r_hist)
plt.plot(r_dicc[:, 0], label='Recompensa 1')
plt.plot(r_dicc[:, 1], label='Recompensa 2')
plt.title("Recompensas por episodio")
plt.xlabel("Episodio")
plt.ylabel("Recompensas")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

front = np.array(front)
plt.scatter(front[:, 0], front[:, 1], color='red')
plt.title("Frontera de Pareto aproximada")
plt.xlabel("Objetivo 1")
plt.ylabel("Objetivo 2")
plt.grid(True)
plt.tight_layout()
plt.show()
